In [1]:
import logging
import os
import sys
import uuid
from contextlib import closing

import yaml
from snowflake.connector import connect

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
stream = logging.StreamHandler()
logger.addHandler(stream)

In [2]:
from snowflake_api import *

In [3]:
from utils import *

In [ ]:
sf_config = get_app_config({})

In [ ]:
base_name = sf_config["subject"]

In [ ]:
[f'{base_name}_{x}' for x in sf_config['extra_databases']]

In [ ]:
sf = SnowflakeApi(sf_config, debug=False)

In [ ]:
sf.run_sql('use role securityadmin')

In [ ]:
sf.run_sql('use role securityadmin;')
sf.create_role('testrole_jschroeder')

In [ ]:
sf.run_sql('use role securityadmin;')
sf.grant_to_role('database', 'itsecurity_test', 'testrole', 'usage')

In [ ]:
sf.grant_to_role('testrole','role','sysadmin')

In [ ]:
base_name = sf_config["subject"]
print("Creating warehouse")
whse = WarehouseApi(sf_config)
for warehouse in sf_config["warehouse_types"]:
    warehouse_name = sf_config["subject"] + warehouse
    wh_create_query = whse.create(whse_name=warehouse_name, **sf_config)
    print(wh_create_query)

print("Creating database and schema(s)")
db = DatabaseApi(sf_config)
prod_db = f"{base_name}_PROD"
prod_db_create = db.create(db_name=prod_db, schemas=["PROJECT", "GLOBAL", "ANALYSIS"])
print(prod_db_create)
for database in sf_config["extra_databases"]:
    db_clone_query = db.clone(
        source_db=prod_db, target_db=f"{base_name}_{database}"
    )
    print(db_clone_query)

In [ ]:
acct = AccountApi(sf_config)
print(acct.create_user(name="TESTUSER"))

In [ ]:
acct = AccountApi(sf_config)
for account in sf_config["warehouse_types"]:
    role_name = f"{sf_config['subject']}{account}"
    user_name = f"{sf_config['subject']}_{account}"
    print(acct.create_user(name=user_name))
    print(acct.create_role(name=role_name))

In [ ]:
for account in sf_config["warehouse_types"]:
    user_name = f"{sf_config['subject']}_{account}"
    print(acct.create_user(name=user_name))
    role_name = f"{sf_config['subject']}{account}"
    print(acct.create_role(name=role_name))
    print(
        acct.grant(
            object_type="role",
            object_name=role_name,
            grantee_type="user",
            grantee=user_name,
        )
    )

In [ ]:
extras = ["DEV", "TEST"]

In [ ]:
extras

In [ ]:
[f"TEST_{x}" for x in extras]

In [ ]:
subject = 'TEST'
primary = 'PROD'
primary_database = f"{subject}_{primary}"
extra_databases = [f"{subject}_{x}" for x in extras]

In [ ]:
extra_databases

In [4]:
sf_config = get_app_config({})
sf_config = set_default_args(sf_config)
project_config = create_db_names(sf_config)

Reading App Config from snowflake_config.yaml
Config values to be used: {'user': 'NEW_PROJECT_SERVICE', 'password': '6339ABBe-2da1-11ea-ba83-b831b5996c28', 'new_user_pwd': 'Strongpassword979', 'azure_account': 'abcsupply.east-us-2.azure', 'subject': 'TEST', 'admin_account': 'WRITER', 'standard_accounts': ['VIEWER'], 'primary_database': 'PROD', 'extra_databases': ['DEV', 'TEST'], 'database_schemas': ['PROJECT', 'GLOBAL', 'ANALYSIS'], 'warehouse_size': 'X-Small', 'auto_suspend': 300, 'auto_resume': True, 'min_cluster_count': 1, 'max_cluster_count': 2, 'initially_suspended': True}


In [6]:
project_config

{'primary_database': 'TEST_PROD',
 'extra_databases': ['TEST_DEV', 'TEST_TEST'],
 'admin_role': 'TESTWRITER',
 'admin_user': 'TEST_WRITER',
 'standard_roles': ['TESTVIEWER'],
 'standard_users': ['TEST_VIEWER'],
 'all_databases': ['TEST_DEV', 'TEST_TEST', 'TEST_PROD'],
 'all_roles': ['TESTVIEWER', 'TESTWRITER'],
 'all_users': ['TEST_VIEWER', 'TEST_WRITER'],
 'all_warehouses': ['TEST_VIEWER', 'TEST_WRITER']}

In [9]:
for x, y, z in zip(
    project_config["all_users"],
    project_config["all_roles"],
    project_config["all_warehouses"],
):
    print("user: ", x, "role: ", y,"warehouse: ", z)

user:  TEST_VIEWER role:  TESTVIEWER warehouse:  TEST_VIEWER
user:  TEST_WRITER role:  TESTWRITER warehouse:  TEST_WRITER


In [ ]:
project_config["admin_role"]

In [ ]:
project_config["all_roles"] = [x for x in project_config["standard_roles"]]

In [ ]:
project_config["all_roles"].append(project_config["admin_role"])

In [ ]:
project_config["all_roles"]

In [ ]:
type(project_config["standard_roles"])

In [ ]:
[x for x in project_config["standard_roles"]].append(project_config["admin_role"])

In [ ]:
project_config = {}

In [ ]:
base_name = sf_config["subject"]

In [ ]:
project_config["primary_database"] = f"{base_name}_{sf_config['primary_database']}"
project_config["extra_databases"] = [
    f"{base_name}_{x}" for x in sf_config["extra_databases"]
]

In [ ]:
project_config